In [ ]:
import scqubits as scq
import numpy as np
import plotly.graph_objects as go

from zcu_tools.analysis.fluxdep import load_result, mA2flx, flx2mA
from zcu_tools.datasaver import load_data

In [ ]:
qub_name = "Test096"

In [ ]:
loadpath = f"../result/{qub_name}/params.json"
_, params, mA_c, period, allows = load_result(loadpath)

if "r_f" in allows:
    r_f = allows["r_f"]
    print(f"r_f = {r_f}")

In [ ]:
def calculate_dispersive(flxs, r_f, g):
    resonator = scq.Oscillator(r_f, truncated_dim=2, id_str="resonator")
    fluxonium = scq.Fluxonium(
        *params, flux=0.5, cutoff=50, truncated_dim=20, id_str="qubit"
    )
    hilbertspace = scq.HilbertSpace([resonator, fluxonium])
    hilbertspace.add_interaction(
        g=g,
        op1=resonator.creation_operator,
        op2=fluxonium.n_operator,
        add_hc=True,
        id_str="q-r coupling",
    )

    def update_hilbertspace(flux):
        fluxonium.flux = flux

    sweep = scq.ParameterSweep(
        hilbertspace,
        {"flxs": flxs},
        update_hilbertspace=update_hilbertspace,
        evals_count=40,
        subsys_update_info={"flxs": [fluxonium]},
        labeling_scheme="LX",
    )

    return sweep

# Plot with Onetone

In [ ]:
onetone_path = "../Database/S001/2025/03/Data_0327/Test096_OneTone_flux_020.hdf5"

signals, sp_fpts, sp_mAs = load_data(
    onetone_path, server_ip="005-writeboard", port=4999
)
sp_fpts = sp_fpts * 1e-9  # to GHz
sp_mAs = sp_mAs * 1e3  # to mA

In [ ]:
r_f = 7.52994
sp_flxs = mA2flx(sp_mAs, mA_c, period)

In [ ]:
flxs = np.linspace(sp_flxs.min(), sp_flxs.max(), 501)
mAs = flx2mA(flxs, mA_c, period)

g = 0.085
sweep = calculate_dispersive(flxs, r_f, g)

In [ ]:
evals = sweep["evals"].toarray()

flx_idxs = np.arange(len(flxs))
idx_00 = sweep.dressed_index((0, 0)).toarray()
idx_10 = sweep.dressed_index((1, 0)).toarray()
idx_01 = sweep.dressed_index((0, 1)).toarray()
idx_11 = sweep.dressed_index((1, 1)).toarray()

rf_0 = evals[flx_idxs, idx_10] - evals[flx_idxs, idx_00]
rf_1 = evals[flx_idxs, idx_11] - evals[flx_idxs, idx_01]

In [ ]:
fig = go.Figure()

# Add the signal as a heatmap
fig.add_trace(
    go.Heatmap(
        z=np.abs(signals),
        x=sp_mAs,
        y=sp_fpts,
        colorscale="Viridis",
        showscale=False,  # Disable the color bar
    )
)

# Add the qubit at 0 and 1 as line plots
fig.add_trace(
    go.Scatter(
        x=mAs,
        y=rf_0,
        mode="lines",
        line=dict(color="blue"),
        name="ground",
    )
)
fig.add_trace(
    go.Scatter(
        x=mAs,
        y=rf_1,
        mode="lines",
        line=dict(color="red"),
        name="excited",
    )
)

# plot a dash hline to indicate the 0 point, also add a xaxis2 to show mA
fig.add_scatter(
    x=sp_mAs,
    y=np.full_like(sp_mAs, r_f),
    xaxis="x2",
    line=dict(color="black", dash="dash"),
    name="origin",
)
mAs_ticks = sp_mAs[:: max(1, len(sp_mAs) // 20)]
flxs_ticks = sp_flxs[:: max(1, len(sp_flxs) // 20)]
fig.update_layout(
    xaxis2=dict(
        tickvals=mAs_ticks,
        ticktext=[f"{flx:.2f}" for flx in flxs_ticks],
        matches="x1",
        overlaying="x1",
        side="top",
        title_text="$Φ_{ext}/Φ_0$",
    )
)

# Update layout
fig.update_layout(
    xaxis_title="mA",
    yaxis_title="Frequency (GHz)",
    legend_title=f"g = {g:.3f} GHz",
    margin=dict(l=0, r=0, t=30, b=0),
)

fig.update_yaxes(range=[sp_fpts.min(), sp_fpts.max()])

fig.show()

In [ ]:
fig.write_html(f"../result/{qub_name}/web/dispersive.html", include_plotlyjs="cdn")
fig.write_image(
    f"../result/{qub_name}/image/dispersive.png", format="png", width=800, height=400
)